# Indice espectrales NDSI Analisis en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP

In [2]:
import os

In [3]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\Indices"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [4]:
import geopandas as gpd

In [5]:
# cargar un shp
zona = gpd.read_file("Zona_Nieve_Geo.shp")
zona

,id,geometry
0,1,"POLYGON ((-77.80787 -8.76092, -77.47329 -8.763..."


In [6]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [7]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [8]:
geometria = ee_zona.geometry()
Map.centerObject(geometria, 10)

In [9]:
# Visualizar en el mapa
Map.addLayer(ee_zona, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-8.907880660247429, -77.65415567840776], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion de imagenes

In [10]:
# Seleccion de la imagen Landsat 5 - 1987
img_L5 = ee.Image("LANDSAT/LT05/C01/T1_SR/LT05_008066_19870531")\
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
                    ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
            .multiply(0.0001)\
            .reproject(crs="EPSG:32718", scale = 30)\
            .clip(ee_zona)

In [11]:
# Seleccion de la imagen Landsat 8 - 2020
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_008066_20200610")\
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                    ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
            .multiply(0.0001)\
            .reproject(crs="EPSG:32718", scale = 30)\
            .clip(ee_zona)

## Consultar nombre bandas y proyección

In [12]:
# Nombre de las bandas
print("Nombre bandas L5: ",img_L5.bandNames().getInfo())
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())

Nombre bandas L5:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [13]:
# Consultar Proyeccion
print("Proyeccion L5: ",img_L5.projection().getInfo())
print("Proyeccion L8: ",img_L8.projection().getInfo())

Proyeccion L5:  {'type': 'Projection', 'crs': 'EPSG:32718', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32718', 'transform': [30, 0, 0, 0, -30, 0]}


In [14]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [15]:
# Visualizar mapa
Map.addLayer(img_L5, viz_es, name = "Landsat5 RS 1987")
Map.addLayer(img_L8, viz_es, name = "Landsat8 RS 2020")
Map

Map(bottom=137874.0, center=[-8.892669730827384, -77.50944581007508], controls=(WidgetControl(options=['positi…

## Determinar NDSI

In [16]:
# Funcion para NDSI
def Calc_NDSI(image):
    ndsi = image.expression("float(green - swir1)/(green + swir1)", {
        'green': image.select('GREEN'),
        'swir1': image.select('SWIR1')
    })
    return ndsi.rename("NDSI")

In [17]:
# Calcular indices con funcion
ndsi_L5 = Calc_NDSI(img_L5)
ndsi_L8 = Calc_NDSI(img_L8)

In [18]:
# Simbologia de NDSI
viz_ndsi = {
    'palette': ['fff947', 'b9fa3e', '5ce5d3', '5172f4', '001334'],
    'min': -0.5,
    'max': 0.8,
    'bands': "NDSI"
}

In [19]:
# Visualizar en el mapa
Map.addLayer(ndsi_L5, viz_ndsi, name = "L5 NDSI")
Map.addLayer(ndsi_L8, viz_ndsi, name = "L8 NDSI")
Map

Map(bottom=1100420.0, center=[-8.813153606144143, -77.69659996032715], controls=(WidgetControl(options=['posit…

## Seleccionar Nieve

In [20]:
# Seleccionar Nieve Landsat 5 - 1987
#  NDSI >= 0.4 es nieve
ndsi_L5_Nieve = ndsi_L5.gte(0.3)
ndsi_L8_Nieve = ndsi_L8.gte(0.4)

## Enmascaramiento de nieve

In [21]:
# Enmascarar NDSI con el nieve
ndsi_L5_mascara = ndsi_L5.updateMask(ndsi_L5_Nieve)
ndsi_L8_mascara = ndsi_L8.updateMask(ndsi_L8_Nieve)

In [22]:
# Visualizar 
Map.addLayer(ndsi_L5_mascara, viz_ndsi, name = "L5 NDSI Mask")
Map.addLayer(ndsi_L8_mascara, viz_ndsi, name = "L8 NDSI Mask")
Map

Map(bottom=550547.0, center=[-8.876591492156717, -77.64947891235353], controls=(WidgetControl(options=['positi…

In [23]:
# Simbologia para cada cambio
viz_ndsi1 = {"min": -0.5,
             "max" : 0.8,
             "palette" : "red"
}

viz_ndsi2 = {"min": -0.5,
             "max" : 0.8,
             "palette" : "blue"
}

In [24]:
# Visualizar 
Map.addLayer(ndsi_L5_mascara, viz_ndsi1, name = "L5 NDSI Mask")
Map.addLayer(ndsi_L8_mascara, viz_ndsi2, name = "L8 NDSI Mask")
Map

Map(bottom=1101530.0, center=[-9.001399723428355, -77.68973350524904], controls=(WidgetControl(options=['posit…

## Exportar Imagen

In [25]:
import os
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [26]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [27]:
# Exportar imagen directorio
geemap.ee_export_image(ndsi_L5_mascara, filename="ndsi_L5_mascara.tif", scale=30, region=geometria, file_per_band=False)
geemap.ee_export_image(ndsi_L8_mascara, filename="ndsi_L8_mascara.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\ndsi_L5_mascara.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\ndsi_L8_mascara.tif


In [ ]:
# Exportar en google drive
geemap.ee_export_image_to_drive(ndsi_L5_mascara, description='ndsi_L5_mascara', folder='GEE_Python', region=geometria, scale=30)
geemap.ee_export_image_to_drive(ndsi_L8_mascara, description='ndsi_L8_mascara', folder='GEE_Python', region=geometria, scale=30)